In [1]:
import numpy as np
import stim
from quits.qldpc_code import *
from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX
from quits.decoder import sliding_window_bposd_circuit_mem
from quits.simulation import get_stim_mem_result


## Load QLDPC codes and circuits

### Hypergraph product (HGP) code

Load the parity check matrix of the classical LDPC code stored in '../parity_check_matrices' directory.  
The parity check matrix `h` is found from `quits.ldpc_utility.generate_ldpc`, following the method in [A. Grospellier, *Constant time decoding of quantum expander codes and application to fault-tolerant quantum computation* (2019)].

In [2]:
# Load the parity check matrix of the classical code that is used to construct the hgp code
h = np.loadtxt('../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt', dtype=int)
print('Shape of classical code\'s parity check matrix: ', h.shape)

Shape of classical code's parity check matrix:  (9, 12)


In [3]:
code = HgpCode(h, h)         # Define the HgpCode object
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).    # Build the Tanner graph and assign directions to its edges. 
                             # For this specific h, seed=22 gives a circuit with entangling depth 8. 
num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

code.verify_css_logicals()   # Verify that the logical operators are valid

# data qubits:  225  # logical qubits:  9
# z-check qubits:  108  # x-check qubits:  108
# layers of entangling gates:  8


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 108,
 'rank_hx': 108,
 'rank_lz': 9,
 'rank_lx': 9,
 'k_expected': 9,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 117,
 'rank_hx_plus_lx': 117,
 'dim_ker_hz': 117,
 'dim_ker_hx': 117,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'pairing_rank': 9,
 'pairing_is_identity': True,
 'ok': True}

For the HGP code, logical operators are written in the "canonical form" (see arXiv:2204.10812)
$$L^z_{ik_1+j} = e_i \otimes L^1_j, \quad L^x_{ik_1+j} = L^2_i \otimes e_j$$
where $L^1_j$ ($L^2_i$) is the $j$($i$)-th logical operator of the 1st (2nd) classical code.  

In [4]:
for i in range(code.lz.shape[0]):
    print('Z%d : '%i, np.where(code.lz[i,:]==1)[0], ', X%d : '%i, np.where(code.lx[i,:]==1)[0])

Z0 :  [111 113 114 115 116 117] , X0 :  [ 45  69  81  93 105 117]
Z1 :  [108 109 113 114 116 118] , X1 :  [ 46  70  82  94 106 118]
Z2 :  [110 112 113 114 116 119] , X2 :  [ 47  71  83  95 107 119]
Z3 :  [123 125 126 127 128 129] , X3 :  [  9  21  69  81 105 129]
Z4 :  [120 121 125 126 128 130] , X4 :  [ 10  22  70  82 106 130]
Z5 :  [122 124 125 126 128 131] , X5 :  [ 11  23  71  83 107 131]
Z6 :  [135 137 138 139 140 141] , X6 :  [ 33  57  69  81 105 141]
Z7 :  [132 133 137 138 140 142] , X7 :  [ 34  58  70  82 106 142]
Z8 :  [134 136 137 138 140 143] , X8 :  [ 35  59  71  83 107 143]


In [5]:
p = 1e-3           # physical error rate
num_rounds = 10    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

idle_error = p     # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
sqgate_error = p   # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
tqgate_error = p   # Float (DEPOLARIZE2 error with rate p) or tuple of 15 floats (pix, ..., pzz)
spam_error = p     # Float

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, idle_error, sqgate_error, tqgate_error, spam_error, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.
## print(circuit)

### Quasi-cyclic lifted product (QLP) code

The base matrix `b` is brought from [Q. Xu et al., arXiv:2308.08648]. Each entry represents the exponent of the monomial. 

In [7]:
lift_size = 16
b = np.array([
    [0, 0, 0, 0, 0],     # [e, e, e, e, e]
    [0, 2, 4, 7, 11],    # [e, x^2, x^4, x^7, x^11]
    [0, 3, 10, 14, 15]   # [e, x^3, x^10, x^14, x^15]
])

In [8]:
code = QlpCode(b, b, lift_size)   # Define the QlpCode object
code.build_circuit(seed=1)          # Build the Tanner graph and assign directions to its edges. 

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

code.verify_css_logicals()   # Verify that the logical operators are valid

# data qubits:  544  # logical qubits:  80
# z-check qubits:  240  # x-check qubits:  240
# layers of entangling gates:  12


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 232,
 'rank_hx': 232,
 'rank_lz': 80,
 'rank_lx': 80,
 'k_expected': 80,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 312,
 'rank_hx_plus_lx': 312,
 'dim_ker_hz': 312,
 'dim_ker_hx': 312,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'pairing_rank': 80,
 'pairing_is_identity': True,
 'ok': True}

In [9]:
p = 1e-3           # physical error rate
num_rounds = 10    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.
## print(circuit)

### Lift-connected surface code

QLP code can be constructed with polynomial entries using `QlpPolyCode`. Each entry is the list of power of the polynomial terms. 

As an example, we generate the circuit of the lift-connected surface code, brought from [J. Old, M. Rispler, and M. Müller, arXiv:2401.02911]. 

In [11]:
lift_size = 5
b = [[[0], [0, 1], []],   # [e, e+x, 0]
     [[], [0], [0, 1]]]   # [0, e, e+x]

In [12]:
code = QlpPolyCode(b, b, lift_size)   # Define the QlpPolyCode object
code.build_circuit(seed=1)           # Build the Tanner graph and assign directions to its edges. 

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

code.verify_css_logicals()   # Verify that the logical operators are valid

# data qubits:  65  # logical qubits:  5
# z-check qubits:  30  # x-check qubits:  30
# layers of entangling gates:  8


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 30,
 'rank_hx': 30,
 'rank_lz': 5,
 'rank_lx': 5,
 'k_expected': 5,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 35,
 'rank_hx_plus_lx': 35,
 'dim_ker_hz': 35,
 'dim_ker_hx': 35,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'pairing_rank': 5,
 'pairing_is_identity': True,
 'ok': True}

In [13]:
p = 1e-3           # physical error rate
num_rounds = 10    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.
## print(circuit)

### Balanced product cyclic (BPC) code

This code is introduced in [R. Tiew & N. P. Breuckmann, arXiv:2411.03302]. 

To precisely match the paper, you should insert $p_2^T$ from arXiv:2411.03302 into $p_2$ here. That is, for the second polynomial, the entries should be `lift_size` minus the powers in arXiv:2411.03302. This is due to the different convention of the parity check matrix we use in QUITS. 

In [15]:
lift_size, factor = 15, 3   # lift_size = factor * q 
p1 = [0, 1, 5]     # e + x + x^5
p2 = [0, 8, 13]    # e + x^8 + x^13, which is TRANSPOSE of e + x^2 + x^7 in arXiv:2411.03302

In [16]:
code = BpcCode(p1, p2, lift_size, factor)  # Define the BpcCode object
code.build_circuit(seed=1)                   # Build the Tanner graph and assign directions to its edges. 

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

code.verify_css_logicals()   # Verify that the logical operators are valid

# data qubits:  90  # logical qubits:  8
# z-check qubits:  45  # x-check qubits:  45
# layers of entangling gates:  8


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 41,
 'rank_hx': 41,
 'rank_lz': 8,
 'rank_lx': 8,
 'k_expected': 8,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 49,
 'rank_hx_plus_lx': 49,
 'dim_ker_hz': 49,
 'dim_ker_hx': 49,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'pairing_rank': 8,
 'pairing_is_identity': True,
 'ok': True}

In [17]:
for i in range(code.lz.shape[0]):
    print('Z%d : '%i, np.where(code.lz[i,:]==1)[0], ', X%d : '%i, np.where(code.lx[i,:]==1)[0])

Z0 :  [ 0  3  6  9 12 15 18 21 24 27] , X0 :  [ 3  4  5  7  9 11 15 17 18 20 21 23 24 26 27 29 30 31 32 33 35 36 37 38
 39 41 60 61 75 76]
Z1 :  [ 1  4  7 10 13 16 19 22 25 28] , X1 :  [16 17 19 20 22 23 25 26 28 29]
Z2 :  [ 0  3  6  9 12 30 33 36 39 42] , X2 :  [ 0  2  4  6  7  8 12 14 16 22 27 29 33 34 35 37 39 41 45 46 75 76]
Z3 :  [ 1  4  7 10 13 31 34 37 40 43] , X3 :  [ 1  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29]
Z4 :  [ 1  2  3  4  5  7  8 12 13 14 15 18 19 20 24 31 32 33 36 37 38 39 40 41
 45 46] , X4 :  [ 0  3  4  7  9 10 11 12 17 18 19 20 21 22 28 29 30 33 34 37 39 40 41 42
 46 76]
Z5 :  [ 1  3  7 15 16 18 21 24 25 30 31 33 34 39 45 47] , X5 :  [ 3  4  5  7  9 11 15 16 17 18 20 21 22 23 24 26 33 34 35 37 39 41 45 46
 75 76]
Z6 :  [ 0  1  2  3  7  8 15 21 24 25 26 30 33 34 35 39 60 61] , X6 :  [ 0  1  2  3  5  8  9 12 13 14 31 33 36 38 40 41 61 76]
Z7 :  [ 3  4  6  7 12 16 18 22 30 31 33 36 39 40 60 62] , X7 :  [ 3  4  5  7  9 11 30 31 32 33 35 36 37 38 39 41 6

In [18]:
p = 1e-3           # physical error rate
num_rounds = 10    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.
## print(circuit)

# Simulate the QEC performance

### Outputs/metrics 
- `pL`: logical error rate per experiment (any logical flip)
- `LFR`: logical failure rate per round, `1 - (1 - pL)^(1/num_rounds)`


In [19]:
num_trials = 100
## Simulate the circuit using Stim. 
detection_events, observable_flips = get_stim_mem_result(circuit, num_trials, seed=1)   # simulate the circuit using Stim

W, F = 5, 3                     # sliding window parameters
max_iter, osd_order = 10, 1     # BP-OSD decoder parameters 

# Perform decoding of the detection_events generated from simulating the circuit. 
# Returns the logical observable flip predicted from decoding. 
logical_pred = sliding_window_bposd_circuit_mem(detection_events, circuit, code.hz, code.lz,\
                                                W, F, max_iter=max_iter, osd_order=osd_order, tqdm_on=True)

# Logical error is recorded whenever logical_pred does not match observable_flips for any logical qubit at any round
pL = np.sum((observable_flips- logical_pred).any(axis=1)) / num_trials
lfr = 1 - (1-pL)**(1/num_rounds)
print('p: %.7f, LFR: %.7f'%(p, lfr))

100%|██████████| 100/100 [00:02<00:00, 34.16it/s]

p: 0.0010000, LFR: 0.0000000


## Example of using a customized inner decoder for sliding-window decoding

In [20]:
#use the BP-OSD decoder from the LDPC package (https://software.roffe.eu/ldpc/) as an example
from quits.decoder import sliding_window_circuit_mem
from ldpc.bposd_decoder import BpOsdDecoder
'''
Note that it is allowed to use a different decoder (Decoder2) for the last decoding window 
than the one used for the other windows (Decoder1).
This feature is useful when you want to: 
   i.e. running BP decoder for all other windows but use BPOSD for the last window to make sure the correction match the syndrome
   
The name of the decoder class is the BpOsdDecoder
The method of this class that takes in a syndrome and returns a correction is "decode"
The parameter of BpOsdDecoder that stores the individual error rate for different error mechanisms is "channel_probs"
An instance of this class is initialized with a parity check matirx, "channel_probs", and the other parameters listed below
'''
#Other paramters that defines the decoders
dict1={'bp_method' : 'product_sum',
        'max_iter' : max_iter,
        'schedule' : 'serial',
        'osd_method' : 'osd_cs',
        'osd_order' : osd_order}
dict2={'bp_method' : 'product_sum',
        'max_iter' : max_iter,
        'schedule' : 'serial',
        'osd_method' :  'osd_cs',
        'osd_order' : osd_order}
# Returns the logical observable flip predicted from decoding. 
logical_pred = sliding_window_circuit_mem(detection_events, circuit, code.hz, code.lz, W, F,
                                          BpOsdDecoder, BpOsdDecoder, dict1, dict2, 
                                          'channel_probs','channel_probs', 'decode', 'decode', tqdm_on=True)

100%|██████████| 100/100 [00:02<00:00, 33.96it/s]
